In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [17]:
train = pd.read_csv('data/train.csv', index_col='Unnamed: 0')
train.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,...,5.0,A,A,A,A,A,A,A,B,D
2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,...,NaN,A,A,C,A,B,A,A,C,A
3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,...,NaN,A,A,A,A,C,A,A,B,A
4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,...,5.0,C,A,A,A,A,D,A,A,A
5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,...,NaN,A,A,A,A,A,A,A,A,A


In [18]:
test = pd.read_csv('data/test.csv')
test.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,43,RS,19,F,1,1,1,3,1,...,NaN,NaN,E,E,H,B,B,C,D,A
1,177f281c68fa032aedbd842a745da68490926cd2,15,PA,24,M,3,2,1,4,1,...,NaN,NaN,B,C,B,A,A,C,D,A
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,29,BA,16,F,2,1,3,0,1,...,NaN,NaN,E,F,G,B,B,A,NaN,D
3,5c356d810fa57671402502cd0933e5601a2ebf1e,41,PR,17,F,1,1,2,0,2,...,40.0,480.0,E,E,E,C,B,B,C,A
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,53,DF,19,F,1,1,1,1,1,...,80.0,720.0,E,E,E,B,B,B,D,A


In [19]:
train = train[list(test.columns) + ['NU_NOTA_MT']]
train.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047,NU_NOTA_MT
1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,43,RS,24,M,1,1,1,4,1,...,520.0,D,D,C,A,A,C,H,A,399.4
2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,23,CE,17,F,3,1,2,0,2,...,580.0,A,A,B,A,A,A,NaN,A,459.8
3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,23,CE,21,F,3,1,3,0,1,...,NaN,D,D,C,A,A,A,NaN,A,NaN
4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,33,RJ,25,F,0,1,1,9,1,...,NaN,H,E,E,C,B,C,F,D,NaN
5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,13,AM,28,M,2,1,1,4,1,...,NaN,E,D,C,A,A,B,F,A,NaN


In [36]:
train_infos = pd.DataFrame({'column': train.columns,
                            'dtype': train.dtypes,
                            'n_missing': train.isna().sum(),
                            'n_unique': train.nunique()}).reset_index().drop('index', axis=1)
train_infos.head()

,column,dtype,n_missing,n_unique
0,NU_INSCRICAO,object,0,13730
1,CO_UF_RESIDENCIA,int64,0,27
2,SG_UF_RESIDENCIA,object,0,27
3,NU_IDADE,int64,0,55
4,TP_SEXO,object,0,2


In [37]:
drop_columns = []
for index, row in train_infos.iterrows():
    if row['n_unique'] == 1:
        drop_columns.append(row['column'])
        
# Drop NU_INSCRICAO as it's just an identification
drop_columns.append('NU_INSCRICAO')
# Drop Q027 as it has more than 50% of values as missing data
drop_columns.append('Q027')
        
train_infos = train_infos[train_infos['n_unique'] != 1]
train_infos = train_infos[train_infos['column'] != 'NU_INSCRICAO']
train_infos = train_infos[train_infos['column'] != 'Q027']

In [38]:
train_infos[train_infos['dtype'] == 'object']

,column,dtype,n_missing,n_unique
2,SG_UF_RESIDENCIA,object,0,27
4,TP_SEXO,object,0,2
35,Q001,object,0,8
36,Q002,object,0,8
37,Q006,object,0,17
38,Q024,object,0,5
39,Q025,object,0,2
40,Q026,object,0,3
42,Q047,object,0,5


In [41]:
train_infos[train_infos['dtype'] == 'int64']

,column,dtype,n_missing,n_unique
1,CO_UF_RESIDENCIA,int64,0,27
3,NU_IDADE,int64,0,55
5,TP_COR_RACA,int64,0,6
6,TP_NACIONALIDADE,int64,0,5
7,TP_ST_CONCLUSAO,int64,0,4
8,TP_ANO_CONCLUIU,int64,0,11
9,TP_ESCOLA,int64,0,4
12,IN_BAIXA_VISAO,int64,0,2
14,IN_SURDEZ,int64,0,2
15,IN_DISLEXIA,int64,0,2
